## (Упрощенная) ERD сущностей в Clickup
![test](ClickupERD.png "Title")

 Команда - основная верхнеуровневая сущность. Название говорит само за себя. Нас интересует исключительно в разрезе team_id, который используется в некоторых эндпойнтах API
 В команде может быть несколько воркспейсов, в которых, в свою очередь, содержатся листы, которые могут быть сгруппированы в папки, но могут быть и прямыми детьми воркспейса.
 Лист - основная сущность, интересующая нас в контексте сборки датасетов, т.к. это коллекция тасков, которые, в свою очередь, содержат всю интересующую нас информацию (дата создания, статус и т.п.). Стоит обратить внимание, что мы можем определить кастомные поля для использования в тасках (например, причина отвала и т.п.). При запросе к API эти поля будут доступны по отдельному ключу custom_fields.
В интерфейсе все это выглядит как-то так:

![Interface](Entities.png "Ent")

In [1]:
import pandas as pd
import json
import ClickUpAPI as cua
import numpy as np
from ast import literal_eval
from math import ceil
from typing import List
from urllib.parse import quote_plus
import os

Первым делом, как и для практически любого API, нам понадобится токен. В Кликапе есть два вида токенов: персональный, который можно получить в секции Apps настроек своего профиля и полноценный authorization code grant OAuth2. Здесь мы будем использовать персональный токен. ![SegmentLocal](PersToken3.gif "segment")

In [2]:
token = os.environ.get("ClickUpToken")

In [3]:
# Проверим, правильно ли API возвращает юзера, от лица которого делаются запросы:
test_class = cua.ClickupClient(token)
res = test_class.get_user()
print(res)

{'id': 8867829, 'username': 'Bogdan Pilyavets', 'email': 'rupilbo@yandex-team.ru', 'color': '#795548', 'profilePicture': None, 'initials': 'BP', 'week_start_day': None, 'global_font_support': False, 'timezone': 'Europe/Moscow'}


In [4]:
# Получим айдишник команды, к которой у нас есть доступ:
team_id = test_class.get_teams(id_only=True)
print(team_id)

4514698


In [5]:
# получим всю инфу про интересующий нас спейс (в данном случае CRM)
space = test_class.get_space_by_name(team_id, name='CRM')
print(space)

{'id': '6849848', 'name': 'CRM', 'private': False, 'statuses': [{'id': 'p6849848_hGuYhhYV', 'status': 'Open', 'type': 'open', 'orderindex': 0, 'color': '#d3d3d3'}, {'id': 'p6849848_aJ5x29At', 'status': 'concept', 'type': 'custom', 'orderindex': 1, 'color': '#FFCC00'}, {'id': 'p6849848_uHca5I69', 'status': 'in progress', 'type': 'custom', 'orderindex': 2, 'color': '#FF540D'}, {'id': 'p6849848_rQtXfgXn', 'status': 'running', 'type': 'custom', 'orderindex': 3, 'color': '#FF00DF'}, {'id': 'p6849848_Yuo3uSOo', 'status': 'review', 'type': 'custom', 'orderindex': 4, 'color': '#A875FF'}, {'id': 'p6849848_e0PVKVnw', 'status': 'Closed', 'type': 'closed', 'orderindex': 5, 'color': '#6bc950'}], 'multiple_assignees': True, 'features': {'due_dates': {'enabled': True, 'start_date': True, 'remap_due_dates': False, 'remap_closed_due_date': False}, 'sprints': {'enabled': False}, 'points': {'enabled': False}, 'custom_items': {'enabled': False}, 'priorities': {'enabled': True, 'priorities': [{'id': '1', '

In [6]:
# получим всю инфу про интересующие нас листы (в данном случае 'Учащиеся' и 'Лиды (родители)')
studs = test_class.get_list_by_name_and_space_id(space_id=space['id'], name='Учащиеся')
pars = test_class.get_list_by_name_and_space_id(space_id=space['id'], name='Лиды (родители)')

In [7]:
tags = test_class.get_tags(space['id'])
tags_names = [tag['name'] for tag in tags['tags']]
tags_to_keep = [quote_plus(tag) for tag in tags_names if tag not in ['тестовый', 'технический']]
tags_to_keep.append('')

In [8]:
#заберем данные по таскам 
tasks = test_class.get_all_tasks(team_id='4514698', include_closed=True, list_ids=[pars['id']])

page=0&list_ids[]=44610695&include_closed=true
page=1&list_ids[]=44610695&include_closed=true
page=2&list_ids[]=44610695&include_closed=true
page=3&list_ids[]=44610695&include_closed=true
page=4&list_ids[]=44610695&include_closed=true
page=5&list_ids[]=44610695&include_closed=true
page=6&list_ids[]=44610695&include_closed=true
page=7&list_ids[]=44610695&include_closed=true
page=8&list_ids[]=44610695&include_closed=true
page=9&list_ids[]=44610695&include_closed=true
page=10&list_ids[]=44610695&include_closed=true
page=11&list_ids[]=44610695&include_closed=true
page=12&list_ids[]=44610695&include_closed=true
page=13&list_ids[]=44610695&include_closed=true
page=14&list_ids[]=44610695&include_closed=true
page=15&list_ids[]=44610695&include_closed=true
page=16&list_ids[]=44610695&include_closed=true
page=17&list_ids[]=44610695&include_closed=true
page=18&list_ids[]=44610695&include_closed=true
page=19&list_ids[]=44610695&include_closed=true
page=20&list_ids[]=44610695&include_closed=true
pa

In [9]:
#поскольку get_all_tasks возвращает нам список nested джейсонов, напишем функцию, которая сложит все это добро в один датафрейм
#подробнее тут https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html
def normalize_wrapper(data:List[dict], **kwargs) -> pd.DataFrame:
    """
    replaces the empty list with a dummy dictionary
    """
    for i in data:
        if not i['tags']:
            i['tags']=[{'name': "fake"}]  #TODO параметризовать ключи
    return pd.json_normalize(data, **kwargs)

def df_getter(data:List[dict], key_getter='tasks', **kwargs) -> pd.DataFrame:
    """
    transforms a list of jsons into a single dataframe using pd.json_normalize
    """
    res = list()
    for chunk in data:
        res.append(normalize_wrapper(chunk[key_getter], **kwargs))
    return pd.concat(res)

In [10]:
par_names = df_getter(tasks,
                      record_path='custom_fields',
                      meta=['id', 'name', 'text_content', 'date_created', 'date_updated', 'date_closed',
                           ['status', 'status'], ['status', 'type'],
                           ['creator', 'email'], 'due_date', 'start_date', 'tags'],
                      meta_prefix='standard_')
#в последней странице почему-то остается только словарь, сейчас нет времени разбираться
missing_list = par_names.loc[par_names['standard_tags']=={'name': 'fake'}]     
par_names.loc[par_names['standard_tags']=={'name': 'fake'}, 'standard_tags'] = pd.Series([[{'name': 'fake'}]]*len(missing_list))

In [11]:
# вытащим теги, чтобы потом выкинуть тестовиков
tags = par_names[['standard_tags', 'standard_id']]
#отберем релевантные столбцы и пивотнем кастомные поля
par_names = par_names.loc[
    par_names['name'].isin([
        'Источник трафика',
        'Канал привлечения',
        'причина отказа',
        'Дата заявки',
        'campaign',
        'content/adset',
        'medium (cpc|cpm)',
        'source (UTM CRM)',
        'причина отказа']), 
        ['name', 'value', 'standard_name', 'standard_id', 'standard_text_content']
        ]
par_names = par_names.pivot(index=['standard_name', 'standard_id', 'standard_text_content'], columns='name', values='value').reset_index()
par_names = par_names.set_index('standard_id')
par_names = par_names.rename(columns={'campaign': 'utm_campaign', 'medium (cpc|cpm)':'utm_medium', 'content/adset': 'utm_content', 'source (UTM CRM)': 'utm_source'})
missing_utms = par_names[par_names['utm_campaign'].isna()]


In [12]:
par_names.head()

name,standard_name,standard_text_content,utm_campaign,utm_content,utm_medium,utm_source,Дата заявки,Источник трафика,причина отказа
standard_id,,,,,,,,,
10hrw71,,Профиль ВК https://vk.com/id\n--данные заявки ...,NaN,NaN,cpc,VK,1630458000000,NaN,7
1nuqc7v,+79026965081,"+79026965081\n19551979zm@gmail.com\n8 класс, н...",NaN,NaN,NaN,NaN,1635037200000,NaN,0
13ntc54,"......,",Телефон: +79885556155\nПочта: kil@gmail.com\nU...,yandex-math_test-offer_leadgen,int_child_skill,cpc,facebook,1626224400000,fb_leads,0
12gmh8p,...ان شاء اللہ,Телефон: 89285451214\nemail: gssh2@mail.ru\nUT...,math_fb_leadgen_lookalike_A/B_test,Lookalike_Yes,cpc,facebook,1631581200000,utm_source=fb;utm_medium=cpc;utm_campaign= mat...,0
xb99um,.русик (FB leads via Zapier),Телефон: +79288796149\nПочта: maya_shamsieva@m...,yandex-math_test-offer_leadgen,int_child_blue,cpc,facebook,1623027600000,fb_leads,0


In [13]:
cust_fields = test_class.get_custom_fields(pars['id'])
cust_df = pd.json_normalize(cust_fields, record_path='fields')
churn = pd.DataFrame.from_records(cust_df.loc[cust_df['name']=='причина отказа', 'type_config.options'].values[0])
churn = churn[['name', 'orderindex']].rename(columns={'name':'churn_reason'})

In [14]:
par_names = par_names.reset_index().merge(right=churn, how='left', left_on='причина отказа', right_on='orderindex')
par_names = par_names.set_index('standard_id')

In [15]:
#разберем строку с utm-метками и превратим ее в 3 столбца
missing_utms['Источник трафика'] = missing_utms['Источник трафика'].fillna("")
missing_utms['utms'] = missing_utms['Источник трафика'].str.findall(r"(?<==)([\w\-_\s]*)")
missing_utms['keys'] = missing_utms['Источник трафика'].str.findall(r"([\w\-_]+)(?==)")
#astype здесь используется как способ конвертнуть пустые и непустые листы в False и True соответственно
mask = missing_utms['keys'].astype(bool)
antimask = missing_utms['keys'].astype(bool) == False
utms = missing_utms.loc[mask, ('utms', 'keys')]
no_utms = missing_utms.loc[antimask, ('Источник трафика', 'utms')]
#Конструируем словарь utm-ок для тех у кого они заполнены
utms['dict'] = utms.apply(lambda x: {x['keys'][i]:x['utms'][i] for i in range(len(x['keys']))}, axis=1)
#Конструируем словарь utm-ок по умолчанию, для тех у кого они не прокинуты
no_utms['dict'] = no_utms.apply(lambda x: {"utm_source":x['Источник трафика']}, axis=1)
#Собираем это воедино, разбиваем на столбцы и джойним к родительскому датафрейму
full_utms = pd.concat([utms['dict'], no_utms['dict']])
stand_index = full_utms.index
full_utms = pd.DataFrame(full_utms.tolist())
full_utms = full_utms.set_index(stand_index)
utm_keys = ['utm_source', 'utm_campaign', 'utm_medium', 'utm_content']
full_utms = full_utms[utm_keys]
#par_names = pd.concat([par_names, full_utms], axis=1)

In [16]:
par_names.loc[par_names['utm_campaign'].isna(), utm_keys] = par_names.loc[par_names['utm_campaign'].isna(), utm_keys].combine_first(full_utms)

In [17]:
#оставим только релевантные столбцы и выкинем каретки
par_names = par_names.reset_index().rename({'index': 'standard_id'}, axis=1)
par_names = par_names[['standard_name', 'standard_id', 'utm_source', 'utm_campaign', 'utm_medium', 'utm_content', 'churn_reason', 'Дата заявки']]
par_names['standard_name'] = par_names['standard_name'].replace(r'\n', ' ', regex=True)
#Конвертнем дату заявки в дату
par_names['Дата заявки'] = (par_names['Дата заявки'].apply(pd.to_datetime, origin='unix', unit='ms') + pd.Timedelta("3 hours")).dt.date
#почистим utm от лишних пробелов и приведем к нижнему регистру
par_names[utm_keys] = par_names[utm_keys].apply(lambda x: x.str.strip())
par_names[utm_keys] = par_names[utm_keys].apply(lambda x: x.str.lower())

In [18]:
#сформируем список тестовых айдишников
tags['standard_tags'] = tags['standard_tags'].astype(str)
throwaway=tags.loc[tags['standard_tags'].str.contains('тестовый|технический'), 'standard_id'].unique()

In [19]:
#получим плоский лист всех родительских айдишников
parent_ids = np.setdiff1d(par_names['standard_id'].unique(), throwaway)
task_ids = list(parent_ids)
history = test_class.get_time_in_status(task_ids)

In [20]:
#поскольку get_time_in_status возвращает нам список словарей,
#в роли ключей у которых выступают родительские айдишники
#соберем это в единый словарь и превратим в датафрейм
flattened = dict()
for task_collection in history:
    flattened.update(task_collection)
flat_history = {task_id:flattened[task_id]['status_history'] for task_id in flattened}
flat_df = pd.DataFrame.from_dict(flat_history, orient='index')
#сейчас у нас столбцы со словарями для истории каждого статуса - пометим их префиксом
columns = ['col'+str(i) for i in range(len(flat_df.columns))]
flat_df.columns = columns
flat_df = flat_df.reset_index()
#разберем вложенные словари в каждой группе столбцов через json_normalize
#и дропнем столбцы со словарями
flat_df = flat_df.where(flat_df.notna(), lambda x: [{}])
for col in columns:
    flat_df = flat_df.join(pd.json_normalize(flat_df[col]).add_prefix(col+'_'))
flat_df = flat_df.drop(columns=columns)
flat_df.head()


,index,col0_status,col0_color,col0_type,col0_orderindex,col0_total_time.by_minute,col0_total_time.since,col1_status,col1_color,col1_type,...,col13_status,col13_color,col13_type,col13_total_time.by_minute,col13_total_time.since,col14_status,col14_color,col14_type,col14_total_time.by_minute,col14_total_time.since
0,10hph70,0. новый лид,#d3d3d3,open,0,23,1630489085181,1. недозвон 1 сутки,#b5bcc2,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10hpmc4,0. новый лид,#d3d3d3,open,0,13,1630489576694,1. недозвон 1 сутки,#b5bcc2,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10hpy37,0. новый лид,#d3d3d3,open,0,13,1630491471014,3.слот встречи назначен,#f9d900,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10hqae0,0. новый лид,#d3d3d3,open,0,14,1630494122104,1. недозвон 1 сутки,#b5bcc2,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10hqc1v,0. новый лид,#d3d3d3,open,0,11,1630494431348,1. недозвон 1 сутки,#b5bcc2,custom,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
#превратим нашу таблицу из широкой в длинную и избавимся от префиксов для столбцов с одинановой сутью
#например, для time_by_minute (время в статусе)
melted_df = flat_df.fillna("").melt(id_vars=['index'])
melted_df['variable'] = melted_df['variable'].str.split("_", n=1).str[1]
#выкинем бесполезные столбцы и сделаем родительские айдишники индексами
melted_df = melted_df[~melted_df['variable'].isin(['orderindex', 'color'])]
melted_df = melted_df.set_index('index')
#соберем список отдельных датафреймов для каждого статуса и схлопнем их в одну таблицу
#NB при итерировании по групбаю он возвращает сначала ключ, а потом датафрейм по этому ключу
to_concat = [y for x,y in melted_df.groupby('variable', as_index=False)]
new_df = pd.concat(to_concat, axis=1)
#дропнем нерелевантные столбцы и переименуем оставшиеся
col_names = list(new_df.iloc[0, [i for i in range(0,len(new_df.columns),2)]])
new_df = new_df.iloc[:,[i for i in range(1,len(new_df.columns),2)]]
new_df.columns = col_names
new_df = new_df.rename(columns={'total_time.by_minute':'min_passed', 'total_time.since': 'start_dt'})
#конвертнем столбец со временем в формат даты
new_df['start_dt'] = new_df['start_dt'].apply(pd.to_datetime, origin='unix', unit='ms') + pd.Timedelta("3 hours")
new_df.head()

,status,min_passed,start_dt,type
index,,,,
10hph70,0. новый лид,23,2021-09-01 12:38:05.181,open
10hpmc4,0. новый лид,13,2021-09-01 12:46:16.694,open
10hpy37,0. новый лид,13,2021-09-01 13:17:51.014,open
10hqae0,0. новый лид,14,2021-09-01 14:02:02.104,open
10hqc1v,0. новый лид,11,2021-09-01 14:07:11.348,open


In [22]:
#получим список всех детей из кликапа
kids = test_class.get_all_tasks(team_id='4514698', include_closed=True, list_ids=[studs['id']])

page=0&list_ids[]=46598432&include_closed=true
page=1&list_ids[]=46598432&include_closed=true
page=2&list_ids[]=46598432&include_closed=true
page=3&list_ids[]=46598432&include_closed=true
page=4&list_ids[]=46598432&include_closed=true
page=5&list_ids[]=46598432&include_closed=true
page=6&list_ids[]=46598432&include_closed=true


In [23]:
#как и родителями, сложим все в один датафрейм
df = df_getter(kids,
               record_path='custom_fields',
               meta=['id', 'name', 'text_content', 'date_created', 'date_updated', 'date_closed',
                    ['status', 'status'], ['status', 'type'],
                    ['creator', 'email'], 'due_date', 'start_date'],
               meta_prefix="standard_")
df.columns

Index(['id', 'name', 'type', 'date_created', 'hide_from_guests', 'required',
       'type_config.fields', 'type_config.field_inverted_name',
       'type_config.linked_subcategory_access',
       'type_config.subcategory_inverted_name', 'type_config.subcategory_id',
       'type_config.options', 'value', 'standard_id', 'standard_name',
       'standard_text_content', 'standard_date_created',
       'standard_date_updated', 'standard_date_closed',
       'standard_status.status', 'standard_status.type',
       'standard_creator.email', 'standard_due_date', 'standard_start_date'],
      dtype='object')

In [24]:
#выкинем тестовых юзеров
df = df[df['standard_status.status']!='тестовый']
df.sample(5)

,id,name,type,date_created,hide_from_guests,required,type_config.fields,type_config.field_inverted_name,type_config.linked_subcategory_access,type_config.subcategory_inverted_name,...,standard_name,standard_text_content,standard_date_created,standard_date_updated,standard_date_closed,standard_status.status,standard_status.type,standard_creator.email,standard_due_date,standard_start_date
978,b2704b5b-55e9-406b-9445-0cdff2661f42,Первая оплата,list_relationship,1637059002273,False,False,"[{'name': 'id ребенка в админке', 'field': 'cf...",NaN,True,Успешные оплаты,...,Арсений Беркович,None,1634464320412,1635181108284,None,первая покупка,custom,nastena-n@yandex-team.ru,1635123600000,None
1073,ec8cfddf-c57c-4971-bacf-fbb61975f3fd,Пары,list_relationship,1633348018326,False,False,"[{'name': 'Преподаватель', 'field': 'cf_683e26...",NaN,True,Пары,...,Татьяна Тюлина (Максим Тюлин папа),,1636212033722,1636212093785,None,первая покупка,custom,shariavova@yandex-team.ru,None,None
35,8c9dfbda-6c96-41e4-a746-6df5b51e2727,Преподаватель списка учащиеся,short_text,1612453867921,False,False,NaN,NaN,NaN,NaN,...,Милана Габеева,,1637340009333,1637340009443,None,первая покупка,custom,elpatsio@yandex-team.ru,None,None
1316,57b23809-f60d-4b04-9978-2d48574f2c3e,Пройденные занятия пакета (старое),short_text,1613133715596,False,False,NaN,NaN,NaN,NaN,...,Аня Курилова,\n,1626534190487,1633349461423,None,новый,open,s-dayana21@yandex-team.ru,None,None
324,57b23809-f60d-4b04-9978-2d48574f2c3e,Пройденные занятия пакета (старое),short_text,1613133715596,False,False,NaN,NaN,NaN,NaN,...,Ева Павлова,\n,1631641606717,1631686581763,None,первая покупка,custom,sadbargmir@yandex-team.ru,None,None


In [25]:
#удалим не нужные нам столбцы
to_drop = ['type_config.default', 'type_config.placeholder', 'type_config.new_drop_down', 'type_config.options',
       'type_config.fields', 'type_config.field_inverted_name',
       'type_config.linked_subcategory_access',
       'type_config.subcategory_inverted_name', 'type_config.subcategory_id']
df = df.drop(columns=df.columns.intersection(to_drop))
df.head()

,id,name,type,date_created,hide_from_guests,required,value,standard_id,standard_name,standard_text_content,standard_date_created,standard_date_updated,standard_date_closed,standard_status.status,standard_status.type,standard_creator.email,standard_due_date,standard_start_date
0,9fb75587-2201-4dc0-9e3d-78f6f2e0c69a,Заявка на допродажу,list_relationship,1633346052564,False,False,NaN,1teehy4,Кирам Шангареев,,1637342759247,1637342859621,1637342859621,отвал (скрыть ученика),closed,shariavova@yandex-team.ru,None,None
1,ec8cfddf-c57c-4971-bacf-fbb61975f3fd,Пары,list_relationship,1633348018326,False,False,NaN,1teehy4,Кирам Шангареев,,1637342759247,1637342859621,1637342859621,отвал (скрыть ученика),closed,shariavova@yandex-team.ru,None,None
2,b2704b5b-55e9-406b-9445-0cdff2661f42,Первая оплата,list_relationship,1637059002273,False,False,NaN,1teehy4,Кирам Шангареев,,1637342759247,1637342859621,1637342859621,отвал (скрыть ученика),closed,shariavova@yandex-team.ru,None,None
3,8c9dfbda-6c96-41e4-a746-6df5b51e2727,Преподаватель списка учащиеся,short_text,1612453867921,False,False,NaN,1teehy4,Кирам Шангареев,,1637342759247,1637342859621,1637342859621,отвал (скрыть ученика),closed,shariavova@yandex-team.ru,None,None
4,57b23809-f60d-4b04-9978-2d48574f2c3e,Пройденные занятия пакета (старое),short_text,1613133715596,False,False,NaN,1teehy4,Кирам Шангареев,,1637342759247,1637342859621,1637342859621,отвал (скрыть ученика),closed,shariavova@yandex-team.ru,None,None


In [26]:
#пивотнем столбец name (каждое уникальное значение из этого столбца становится отдельным столбцом в пивотизированном датафрейме)
#поскольку столбец 'родитель в воронке лидов' в кликапе является ссылкой на другой лист, API возвращает нам этот лист, а пандас
#читает его как питоновский лист. В нашем случае в нем всегда один элемент, поэтому explode не увеличивает кол-во строк в датафрейме
#а просто избавляет нас от листа в cтолбце, превращая ее в столбец словарей
df2 = df.copy()
df2 = df2.pivot(index='standard_id', columns='name', values='value')
df2 = df2.explode('Родитель в воронке лидов')
df2.head()

name,Shared with me,Заявка на допродажу,Инциденты,Пары,Первая оплата,Преподаватель списка учащиеся,Прогулы,Пройденные занятия пакета (старое),Расписание,Родитель в воронке лидов,Согласие на видеозапись,Цель ученика и сроки достижения цели,класс,комментарий для преподавателя,купленные пакеты,первое занятие
standard_id,,,,,,,,,,,,,,,,
10htvj3,NaN,"[{'id': '1g5gmfa', 'name': 'Борис Севастьянов ...",NaN,NaN,NaN,NaN,"[{'id': '118uqak', 'name': 'Борис Севастьянов'...",NaN,NaN,"{'id': 'z9w1ab', 'name': 'Ксения Екимова', 'st...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,NaN,NaN,"[{'id': '10htu81', 'name': 'Борис Севастьянов'...",NaN
118tu1c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'id': 'wkyr6z', 'name': 'Елена Маклина', 'sta...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,NaN,NaN,"[{'id': '1gbdutu', 'name': 'Маклина Елена', 's...",NaN
118tz1t,NaN,"[{'id': '1q2qrd2', 'name': 'Александр Крок', '...",NaN,NaN,NaN,Екатерина Пинаева,"[{'id': '1fu5xur', 'name': 'Анна Крок', 'statu...",NaN,"вт 20:00, пт 19:00","{'id': '118pbfa', 'name': 'Анастасия Крок', 's...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,7,NaN,"[{'id': '118ty1k', 'name': 'Александр Крок', '...",NaN
118u1dz,NaN,NaN,NaN,NaN,NaN,Георгий Поседко,NaN,NaN,ВС. 12:00 – 13:00,"{'id': 'z9watm', 'name': 'Екатерина Александро...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,NaN,NaN,"[{'id': '118u1a2', 'name': 'Екатерина Александ...",NaN
118uqam,NaN,"[{'id': '1phpdpr', 'name': 'Dead Heat Ирина Пр...",NaN,NaN,NaN,Георгий Поседко,"[{'id': '1g006x5', 'name': 'Андрей Преображенс...",NaN,ЧТ. 18:00 – 19:00 СБ. 15:00 – 16:00,"{'id': 'yyd589', 'name': 'Ирина Преображенская...",[8472ccaa-0c6e-4537-b71c-f2345ec26b11],NaN,7,NaN,"[{'id': '118up13', 'name': 'Андрей Преображенс...",NaN


In [27]:
#читаем наш столбец со словарями в отдельный датафрейм, добавляем префикс к именам столбцов
#чтобы избежать конфликтов при последующем мердже
parent = df2['Родитель в воронке лидов'].apply(pd.Series).add_prefix("par_")
parent = parent[['par_id', 'par_name']]
parent.head()

,par_id,par_name
standard_id,,
10htvj3,z9w1ab,Ксения Екимова
118tu1c,wkyr6z,Елена Маклина
118tz1t,118pbfa,Анастасия Крок
118u1dz,z9watm,Екатерина Александрова
118uqam,yyd589,Ирина Преображенская Dead Heat


In [58]:
# проделываем то же самое для столбца со ссылкой на купленные пакеты по новому флоу
new_flow_purch = (
    df2['Первая оплата'].dropna()
                        .explode()
                        .apply(pd.Series)
                        .add_prefix("purch_")
    )
new_flow_purch = new_flow_purch[['purch_id']].reset_index()
new_flow_purch.head()

,standard_id,purch_id
0,1dvc0b9,1td6azv
1,1g02567,1tjw6x7
2,1jtbh9p,1tjvgh8
3,1kvpwjv,1tycum6
4,1m1kc26,1tjtudk


In [40]:
# получаем инфу о списке покупок по новому флоу
purch_list_new = test_class.get_list_by_name_and_space_id(space_id=space['id'], name='Успешные оплаты')
print(purch_list_new)

{'id': '156475141', 'name': 'Успешные оплаты', 'orderindex': 0, 'status': None, 'priority': None, 'assignee': None, 'task_count': 32, 'due_date': None, 'start_date': None, 'folder': {'id': '90393097', 'name': 'hidden', 'hidden': True, 'access': True}, 'space': {'id': '6849848', 'name': 'CRM', 'access': True}, 'archived': False, 'override_statuses': False, 'permission_level': 'create'}


In [41]:
# забираем все таски из этого списка
new_flow_purchases = test_class.get_all_tasks(team_id='4514698', include_closed=True, list_ids = [purch_list_new['id']])

page=0&list_ids[]=156475141&include_closed=true
page=1&list_ids[]=156475141&include_closed=true


In [55]:
# Соберем инфу в датафрейм и оставим релевантные столбцы
new_flow_df = df_getter(new_flow_purchases, record_path='custom_fields', meta=['id', ['status', 'status']],meta_prefix='standard_')
new_flow_df = new_flow_df[(new_flow_df['name']=='id ребенка в админке') & (new_flow_df['standard_status.status']!='тестовый')]
new_flow_df = new_flow_df[['standard_id', 'value']].rename(columns={'value': 'praktikum_id'})
new_flow_df.head()

,standard_id,praktikum_id
9,1tyg0w9,13332515
23,1tyfznc,13332375
37,1tyfyxj,13332342
51,1tyf4xu,12910602
65,1tycum6,12768184


In [59]:
#проделываем то же самое для столбца со ссылкой на купленные пакеты по старому флоу
packages = df2['купленные пакеты'].explode().apply(pd.Series).add_prefix("purch_")
packages = packages[['purch_id']].reset_index()
packages.head()

,standard_id,purch_id
0,10htvj3,10htu81
1,118tu1c,1gbdutu
2,118tz1t,118ty1k
3,118u1dz,118u1a2
4,118uqam,118up13


In [29]:
#получаем инфу о списке покупок в кликапе
purch_list = test_class.get_list_by_name_and_space_id(space_id=space['id'], name='Заявки на покупку')
print(purch_list)

{'id': '48853634', 'name': 'Заявки на покупку', 'orderindex': 0, 'status': None, 'priority': None, 'assignee': None, 'task_count': 928, 'due_date': None, 'start_date': None, 'folder': {'id': '23627065', 'name': 'hidden', 'hidden': True, 'access': True}, 'space': {'id': '6849848', 'name': 'CRM', 'access': True}, 'archived': False, 'override_statuses': True, 'permission_level': 'create'}


In [30]:
#забираем все таски из этого списка
purchases = test_class.get_all_tasks(team_id='4514698', include_closed=True, list_ids = [purch_list['id']])

page=0&list_ids[]=48853634&include_closed=true
page=1&list_ids[]=48853634&include_closed=true
page=2&list_ids[]=48853634&include_closed=true
page=3&list_ids[]=48853634&include_closed=true
page=4&list_ids[]=48853634&include_closed=true
page=5&list_ids[]=48853634&include_closed=true
page=6&list_ids[]=48853634&include_closed=true
page=7&list_ids[]=48853634&include_closed=true
page=8&list_ids[]=48853634&include_closed=true
page=9&list_ids[]=48853634&include_closed=true
page=10&list_ids[]=48853634&include_closed=true


In [61]:
#и внвь соберем эту инфу в плоский датафрейм, выкинем тесовых и оставим только релевантные столбцы
purch_df = df_getter(purchases, record_path='custom_fields', meta=['id', ['status', 'status']],meta_prefix='standard_')
purch_df = purch_df[(purch_df['name']=='Ссылка на ребенка в админке') & (purch_df['standard_status.status']!='тестовый')][['value', 'standard_id']].dropna()
#вытащим praktikum_id из юрла
purch_df['praktikum_id'] = purch_df['value'].str.split("/").str[-1]
purch_df = purch_df[['standard_id', 'praktikum_id']]
purch_df.head()

,standard_id,praktikum_id
3,1t7dq3f,13279314
13,1t7dngq,11869105
23,1t7dec6,13282216
33,1t7d51a,13281594
43,1t7crta,13280887


In [63]:
# склеим датафреймы старого и нового флоу
packages_full = pd.concat([packages, new_flow_purch], ignore_index=True)
purch_df_full = pd.concat([purch_df, new_flow_df], ignore_index=True)

In [64]:
#сопоставим айдишники детей с их айдишниками в практикуме
kids_links = packages_full.merge(purch_df_full, how='left', left_on='purch_id', right_on='standard_id')
kids_links = kids_links[['purch_id', 'praktikum_id', 'standard_id_x']].groupby('standard_id_x')['praktikum_id'].last()
kids_links.head()

standard_id_x
10htvj3    12363984
118tu1c    12760158
118tz1t    12391974
118u1dz    12392531
118uqam    12394280
Name: praktikum_id, dtype: object

In [65]:
#теперь сджойним детей с их родителями
full_lineage = parent.merge(kids_links, how='left', left_index=True, right_index=True)
full_lineage.sample(5)

,par_id,par_name,praktikum_id
standard_id,,,
afwd68,15xx56q,Олеся Карабанова,11442828
12axqdj,118tp2g,Наталья Исаева,12449650
12pfe1d,125c2yg,Назгуль Токтоналиева,12504255
1me3qgu,1hwrwca,Татьяна Дворецкая,12900031
nzcct6,139md6q,Костючкова Надежда (Даня),11128723


In [66]:
#наконец, смерджим все это воедино с датафреймом родительских атрибутов
fin_df = new_df.reset_index().merge(full_lineage, how='left', left_on='index', right_on='par_id')
fin_df = fin_df[fin_df['type']!=""]
fin_df = fin_df.merge(par_names, how='left', left_on='index', right_on='standard_id')
fin_df.sample(10)

,index,status,min_passed,start_dt,type,par_id,par_name,praktikum_id,standard_name,standard_id,utm_source,utm_campaign,utm_medium,utm_content,churn_reason,Дата заявки
26797,1m1na9j,6.нужна перезапись,837.0,2021-10-11 20:34:36.024,custom,NaN,NaN,NaN,Артём Пидодня,1m1na9j,facebook,math_fb_leadgen_kids&teachers - men,cpc,fb_inattention_men,дорого - бесплатно хочу,2021-10-09
4940,1t7jnvp,0. новый лид,7,2021-11-16 15:13:26.966,open,NaN,NaN,NaN,Елена Девяшина,1t7jnvp,facebook,math_fb_leadgen_grow_up,cpc,grow_up,NaN,2021-11-16
12556,khc4x5,1. недозвон 2 сутки,1277.0,2021-06-26 13:36:23.809,custom,NaN,NaN,NaN,Ирина Егорова,khc4x5,facebook,yandex-math_test-offer_leadgen,cpc,NaN,недозвон,2021-05-27
15936,1j8eqc6,2. отказ после контакта,84284.0,2021-09-22 10:37:30.947,custom,NaN,NaN,NaN,игорь комендантов,1j8eqc6,facebook,math_fb_leadgen_kids&teachers - men,cpc,fb_teacher_men,не ЦА - класс,2021-09-22
14143,118kf28,7. ждем оплаты,967.0,2021-09-14 16:29:21.714,custom,118kf28,Анна Федоренко (папа Алексей Федоренко),12590354,Анна Федоренко (папа Алексей Федоренко),118kf28,facebook,math_fb_traffic_september_offer,cpc,offer2_yellow_math_individual_program,NaN,2021-09-02
8176,12w92xe,1. недозвон 1 сутки,1.0,2021-09-18 11:27:52.119,custom,NaN,NaN,NaN,Светлана Ярцева,12w92xe,facebook,math_fb_leadgen_innatention_lal,cpc,fb_innatention_lal_2.0,не ЦА - класс,2021-09-18
20886,12b4pkw,2.перезвонить на неделе,8776.0,2021-09-14 11:10:19.736,custom,NaN,NaN,NaN,Kристина,12b4pkw,facebook,math_fb_leadgen_homework,cpc,fb_homework_lal,недозвон,2021-09-14
29029,11bjdrg,9. есть оплата и запись,106658.0,2021-07-13 13:43:53.377,custom,11bjdrg,Елена Кренцель (Иосиф),11157305,Елена Кренцель (Иосиф),11bjdrg,facebook,yandex-math_test-offer_leadgen,cpc,int_child_everyone,отвал после первого месяца,2021-07-08
3295,1nf9ny3,0. новый лид,135,2021-10-21 08:38:00.659,open,NaN,NaN,NaN,Светлана Коровкина,1nf9ny3,facebook,math_fb_leadgen_kids&teachers - lal,cpc,fb_inattention,NaN,2021-10-21
22818,1nn5wn8,1. недозвон (нет контакта),30736.0,2021-10-29 15:06:17.193,custom,NaN,NaN,NaN,Зарина Цибирова,1nn5wn8,facebook,math_fb_leadgen_kids&teachers - lal,cpc,fb_study_begin,недозвон,2021-10-23


In [67]:
#добавим мэппинг статусов из файла
statuses = pd.read_excel('mappings.xlsx', engine='openpyxl', sheet_name='statuses', index_col=1)
churn_reasons = pd.read_excel('mappings.xlsx', engine='openpyxl', sheet_name='dropouts')
fin_df = fin_df.merge(statuses, left_on='status', right_index=True, how='left')
fin_df = fin_df.merge(churn_reasons, left_on='churn_reason', right_on='detailed', how='left').drop(columns=['detailed'])
fin_df.head(10)

,index,status,min_passed,start_dt,type,par_id,par_name,praktikum_id,standard_name,standard_id,...,utm_campaign,utm_medium,utm_content,churn_reason,Дата заявки,этап,Созвон,Бронирование,Встреча,aggregated
0,10hph70,0. новый лид,23,2021-09-01 12:38:05.181,open,NaN,NaN,NaN,Кати Долгова,10hph70,...,math_fb_leadgen_lookalike_a/b_test,cpc,lookalike_no,недозвон,2021-09-01,0 заявка,0.0,0.0,0.0,Недозвон
1,10hpmc4,0. новый лид,13,2021-09-01 12:46:16.694,open,NaN,NaN,NaN,Татьяна Гулько,10hpmc4,...,math_fb_leadgen_kids&teachers,cpc,fb_study_begin,недозвон,2021-09-01,0 заявка,0.0,0.0,0.0,Недозвон
2,10hpy37,0. новый лид,13,2021-09-01 13:17:51.014,open,10hpy37,Оксана Маришкина,12484377,Оксана Маришкина,10hpy37,...,math_yandex_search,cpc,NaN,NaN,2021-09-01,0 заявка,0.0,0.0,0.0,NaN
3,10hqae0,0. новый лид,14,2021-09-01 14:02:02.104,open,10hqae0,Наталья Цибульник,12635010,Наталья Цибульник,10hqae0,...,math_fb_leadgen_lookalike_a/b_test,cpc,lookalike_yes,возврат средств,2021-09-01,0 заявка,0.0,0.0,0.0,Отвал после использования
4,10hqc1v,0. новый лид,11,2021-09-01 14:07:11.348,open,NaN,NaN,NaN,Нигара Аркин,10hqc1v,...,math_fb_leadgen_kids&teachers - lal,cpc,fb_study_begin,недозвон,2021-09-01,0 заявка,0.0,0.0,0.0,Недозвон
5,10hqkbr,0. новый лид,15,2021-09-01 14:37:45.110,open,NaN,NaN,NaN,Елена Лачугина,10hqkbr,...,math_fb_leadgen_kids&teachers - lal,cpc,fb_inattention,уже неакт. - отстаньте,2021-09-01,0 заявка,0.0,0.0,0.0,Уже неактуально
6,10hqrdm,0. новый лид,1,2021-09-01 14:58:44.035,open,NaN,NaN,NaN,Елена,10hqrdm,...,math_fb_leadgen_kids&teachers - lal,cpc,fb_inattention,недозвон,2021-09-01,0 заявка,0.0,0.0,0.0,Недозвон
7,10hqwmp,0. новый лид,3,2021-09-01 15:08:59.467,open,NaN,NaN,NaN,Валентина Кузьмич,10hqwmp,...,math_fb_leadgen_august,cpc,math_august,уже неакт. - передумали,2021-09-01,0 заявка,0.0,0.0,0.0,Уже неактуально
8,10hr1ad,0. новый лид,9,2021-09-24 16:12:12.633,open,10hr1ad,Ленара Дегтярева,12731723,Ленара Дегтярева,10hr1ad,...,math_fb_leadgen_kids&teachers,cpc,fb_teacher,NaN,2021-09-01,0 заявка,0.0,0.0,0.0,NaN
9,10hrnnr,0. новый лид,103,2021-09-01 16:32:24.559,open,10hrnnr,Елена Редько,12449405,Елена Редько,10hrnnr,...,NaN,NaN,NaN,NaN,2021-09-01,0 заявка,0.0,0.0,0.0,NaN


In [68]:
#создадим отдельные столбцы с датой созвона, брони и встречи
fin_df['call'] = fin_df['start_dt'].where(fin_df['Созвон']==1, np.NaN).groupby(fin_df['index']).transform('min')
fin_df['meeting'] = fin_df['start_dt'].where(fin_df['Встреча']==1, np.NaN).groupby(fin_df['index']).transform('min')
fin_df['booking'] = fin_df['start_dt'].where(fin_df['Бронирование']==1, np.NaN).groupby(fin_df['index']).transform('min')
#выкинем лишние столбцы
fin_df = fin_df.drop(columns=['Созвон', 'Встреча', 'par_name', 'index', 'par_id'])
fin_df['lead_date'] = fin_df.groupby('standard_id')['start_dt'].transform('min')
fin_df['lead_date'] = fin_df['Дата заявки'].combine_first(fin_df['lead_date'])
#конвертнем дату заявки в datetime, предварительно проверив на пустоту
if len(fin_df.loc[fin_df['Дата заявки'].isna(), 'lead_date']) > 0:
    fin_df.loc[fin_df['Дата заявки'].isna(), 'lead_date'] = fin_df[fin_df['Дата заявки'].isna()]['lead_date'].apply(pd.to_datetime).dt.date
fin_df = fin_df.drop(columns=['Дата заявки', 'Бронирование'])
fin_df = fin_df.rename(columns={'этап': 'etap'})
fin_df.sample(7)

,status,min_passed,start_dt,type,praktikum_id,standard_name,standard_id,utm_source,utm_campaign,utm_medium,utm_content,churn_reason,etap,aggregated,call,meeting,booking,lead_date
16676,1. недозвон 2 сутки,4323.0,2021-10-16 15:19:18.089,custom,NaN,Жанна Терентева,1md3jbb,facebook,math_fb_leadgen_innatention_lal_new,cpc,fb_innatention_lal_2.0,уже неакт. - купили,1 устанавливаем контакт,Уже неактуально,2021-10-19 15:22:24.339,NaT,NaT,2021-10-12
20937,1. недозвон (пингуем нед.),8691.0,2021-09-21 09:21:14.314,custom,NaN,Александр Галченков,12grc6h,facebook,math_fb_leadgen_kids&teachers - men,cpc,fb_inattention_men,недозвон,1 устанавливаем контакт,Недозвон,NaT,NaT,NaT,2021-09-14
11064,2.перезвонить на неделе,1538.0,2021-11-01 09:50:41.832,custom,NaN,Наталья Мизинина,1pqe2kc,facebook,math_fb_leadgen_video,cpc,math_fb_lal+match_video,NaN,2 выясняем цели,NaN,2021-11-01 09:50:41.832,2021-11-10 16:41:56.827,2021-11-02 11:29:30.891,2021-11-01
3701,0. новый лид,500,2021-10-27 01:01:40.979,open,NaN,Елена Овсянникова,1p658x0,facebook,math_fb_leadgen_tempo,cpc,math_fb_lal+match,недозвон,0 заявка,Недозвон,2021-10-27 09:21:55.615,NaT,2021-10-27 09:21:55.615,2021-10-27
9188,2. отказ после контакта,62641.0,2021-10-07 11:20:31.573,custom,NaN,Ольга Киреева,1jz4v24,,,,,недозвон,2 выясняем цели,Недозвон,2021-10-05 09:15:49.808,NaT,2021-10-05 09:15:49.808,2021-09-27
16138,3.слот встречи назначен,11351.0,2021-10-11 15:32:39.608,custom,NaN,Альфия Гатауллина,1knz76z,facebook,math_fb_leadgen_carousel,cpc,fb_leadgen_carousel_testimonials,уже неакт. - отстаньте,3 бронируем встречу,Уже неактуально,2021-10-09 10:53:05.258,NaT,2021-10-09 10:53:05.258,2021-10-05
11178,3.слот встречи назначен,1190.0,2021-11-03 18:55:45.764,custom,NaN,Бадалова Севинч,1q2x1na,facebook,math_fb_leadgen_november,cpc,concrete_offer_w_33_55_math_edu_and_aud_network,недозвон,3 бронируем встречу,Недозвон,2021-11-03 12:04:03.824,NaT,2021-11-03 12:04:03.824,2021-11-03


In [69]:
fin_df['standard_id'].nunique()

7027

In [70]:
fin_df.to_csv('fin1_df.csv')

# ЗЭ ЭНД